## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [35]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [36]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [37]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [38]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [39]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.919203,0.181923,0.013529,0.127419,0.037550,0.002808,SonyAIBORobotSurface2,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.828961,0.168610,0.021266,0.037572,0.106363,0.002882,SonyAIBORobotSurface2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,0.876180,1.377738,0.011656,0.138769,1.211099,0.015296,SonyAIBORobotSurface2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,(max instances per class)//2,Rsast,ACF&PACF


In [40]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['SonyAIBORobotSurface2', 'GunPoint', 'DistalPhalanxOutlineCorrect',
       'SmoothSubspace', 'DistalPhalanxOutlineAgeGroup', 'ShapeletSim',
       'ItalyPowerDemand', 'SyntheticControl', 'SonyAIBORobotSurface1'],
      dtype=object)

In [41]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [42]:

filter_param=df_result[df_result.rpoint=='100']
filter_param.rpoint.unique()

array(['100'], dtype=object)

In [43]:

filter_param=filter_param[filter_param.nb_per_class=='100']
filter_param.nb_per_class.unique()

array(['100'], dtype=object)

In [44]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,DistalPhalanxOutlineAgeGroup,0.750600,0.721823,0.755396,0.714628,0.705036,0.690647,0.748201,ROCKET
1,DistalPhalanxOutlineCorrect,0.657005,0.723430,0.665459,0.699275,0.741546,0.713768,0.771739,ROCKET
2,GunPoint,0.815556,0.980000,0.857778,0.973333,0.973333,0.977778,1.000000,ROCKET
3,ItalyPowerDemand,0.954001,0.948170,0.955620,0.945254,0.952057,0.949466,0.968902,ROCKET
4,ShapeletSim,0.985185,0.883333,0.985185,0.820370,0.551852,0.883333,1.000000,ROCKET
5,SmoothSubspace,0.937778,0.922222,0.922222,0.926667,0.917778,0.942222,0.980000,ROCKET
6,SonyAIBORobotSurface2,0.855894,0.915005,0.856593,0.879328,0.859741,0.913956,0.916055,ROCKET
7,SyntheticControl,0.935000,0.970000,0.931667,0.968333,0.990000,0.971667,1.000000,ROCKET


In [45]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.831664,0.752720,0.808886,0.834165,0.830541,0.830316
ACF&PACF,0.896216,0.849249,0.870615,0.874843,0.872004,0.873263
Max ACF,0.835663,0.743076,0.812849,0.838455,0.830321,0.838078
Max PACF,0.870380,0.803172,0.850124,0.849230,0.852417,0.848475
None,0.861229,0.793778,0.821339,0.837700,0.821995,0.838799
PACF,0.896193,0.852509,0.868878,0.873791,0.876989,0.873285


In [46]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.832706,0.719289,0.814716,0.843347,0.839583,0.838562
ACF&PACF,0.898033,0.831063,0.866160,0.879593,0.880732,0.880455
Max ACF,0.830986,0.722978,0.815981,0.845764,0.842440,0.840692
Max PACF,0.871718,0.780365,0.850314,0.852691,0.862078,0.856518
None,0.868525,0.759758,0.826409,0.837206,0.844053,0.838266
PACF,0.896377,0.844038,0.868668,0.876926,0.877165,0.878455


#### Time: Subsequence Lenght Method 

In [47]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,
ACF,111.545551,144.720377,36.703966,0.353901,7.361113,0.561044,0.190215,0.700003,91.219709
ACF&PACF,294.637278,356.864070,70.434438,0.729001,146.904885,0.930945,1.326816,2.032898,211.660371
Max ACF,40.968521,47.509385,4.170987,0.361998,2.827787,0.565765,0.176733,0.548111,32.137693
Max PACF,19.410193,21.999060,3.941916,0.387409,66.417229,0.639174,0.245754,0.413045,16.766244
None,18.884042,21.533008,2.339549,0.331860,1.788605,0.523768,0.147382,0.305284,16.088837
PACF,202.722461,237.791324,45.974556,0.711325,140.841731,0.843548,1.022764,1.655537,160.944801


In [48]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,4.508369,2.193618,15.032718,95.979918,41.131245,60.841663
ACF&PACF,10.010423,5.897197,53.998957,254.193136,116.772793,165.730869
Max ACF,0.670638,1.144784,5.059812,31.332334,12.802873,19.303096
Max PACF,0.844306,1.864408,12.650341,23.922677,15.923178,18.544907
None,0.468652,0.943885,2.669634,14.691229,6.118789,8.877618
PACF,6.671876,4.712713,42.404453,181.324799,85.627279,119.833417


In [49]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,6.931851,2.068415,12.984172,95.744418,38.234446,63.517574
ACF&PACF,15.145816,12.908871,45.835739,232.152147,118.267851,181.855050
Max ACF,0.933106,1.029267,4.740326,30.072938,12.668144,20.847373
Max PACF,1.108135,8.973852,10.972347,18.695151,15.710880,18.266955
None,0.630012,0.852939,2.825106,12.522585,6.867428,10.057977
PACF,10.351703,11.568828,34.753159,160.120164,89.401038,134.065862


#### Variance: Subsequence Lenght Method 

In [50]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.008971,0.018017,0.011495,0.012181,0.011139,0.011401
ACF&PACF,0.004603,0.009948,0.008527,0.009203,0.008956,0.009206
Max ACF,0.010991,0.017249,0.012070,0.012168,0.010172,0.012242
Max PACF,0.008432,0.019876,0.008856,0.011346,0.009439,0.010107
None,0.007534,0.018436,0.016212,0.017260,0.017486,0.018230
PACF,0.004375,0.009639,0.008372,0.010048,0.009233,0.009691


In [51]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.008495,0.018731,0.013539,0.012601,0.013102,0.013231
ACF&PACF,0.004466,0.011202,0.008698,0.009068,0.009405,0.009112
Max ACF,0.008628,0.021031,0.013660,0.012486,0.011600,0.011870
Max PACF,0.008505,0.015577,0.011877,0.011945,0.010528,0.012543
None,0.008050,0.020372,0.018698,0.016480,0.018832,0.016943
PACF,0.004379,0.010836,0.009104,0.009344,0.008722,0.009453


#### Another Results

In [52]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,4710,4710,4710
unique,9,6,216
top,SonyAIBORobotSurface2,ACF&PACF,Max ACF: n_random_points= (lenthg ts)//2 nb_in...
freq,720,785,70


In [53]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
DistalPhalanxOutlineAgeGroup,14.336834
DistalPhalanxOutlineCorrect,17.300359
GunPoint,4.770658
ItalyPowerDemand,0.095850
ShapeletSim,7.627945
SmoothSubspace,0.135475
SonyAIBORobotSurface1,0.038871
SonyAIBORobotSurface2,0.188496
SyntheticControl,7.344690


In [54]:
df_result['time'].sum()/(60*60)

51.839176846941314

#### Generate CD Diagram Tunning Hyperparameter

In [55]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()


In [56]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['GunPoint', 'ItalyPowerDemand', 'SmoothSubspace',
       'SonyAIBORobotSurface2'], dtype=object)

In [57]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

In [58]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
Max ACF: n_random_points=1 nb_inst_per_class=1,214.250
ACF: n_random_points=1 nb_inst_per_class=1,214.250
ACF: n_random_points=10 nb_inst_per_class=1,209.000
Max PACF: n_random_points=1 nb_inst_per_class=1,208.375
None: n_random_points=1 nb_inst_per_class=1,206.500
...,...
PACF: n_random_points=30 nb_inst_per_class=50,36.500
PACF: n_random_points=100 nb_inst_per_class=100,35.250
PACF: n_random_points=50 nb_inst_per_class=100,33.750
ACF&PACF: n_random_points=100 nb_inst_per_class=50,31.625


In [59]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [60]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50'], dtype='object')

In [61]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.928991,0.865139,0.910525,0.937815,0.939811,0.944780
1,0.924608,0.768364,0.898312,0.918624,0.926859,0.909377
10,0.923721,0.868657,0.919051,0.939790,0.939651,0.941335
100,0.934857,0.884688,0.924395,0.941349,0.941673,0.946673
30,0.934228,0.886199,0.920385,0.943795,0.937354,0.943178
50,0.925559,0.890218,0.926359,0.943884,0.941349,0.942935


In [62]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.000935,0.005909,0.002099,0.000842,0.000723,0.000693
1,0.001359,0.022783,0.004532,0.002178,0.001955,0.003764
10,0.001556,0.005706,0.002192,0.000832,0.000844,0.000589
100,0.000946,0.005550,0.001781,0.000785,0.000718,0.000603
30,0.000964,0.007255,0.002199,0.000836,0.001052,0.000718
50,0.001389,0.001855,0.001902,0.000617,0.000874,0.000667


##### Overall time complexity

In [63]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
GunPoint,0.000645,0.027058,0.000981,1.145110,1.173907
ItalyPowerDemand,0.000136,0.001336,0.000842,0.009811,0.012150
SmoothSubspace,0.000103,0.002251,0.002864,0.010263,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.000283,0.030471,0.033882


In [64]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,0.268661,0.025665,0.213107,0.522881,0.519338,0.533891
1,0.044270,0.009157,0.016136,0.028492,0.025892,0.026516
10,0.045165,0.013833,0.055648,0.125056,0.128500,0.128659
30,0.118813,0.020914,0.142794,0.346109,0.349427,0.345032
50,0.185372,0.030580,0.229786,0.558682,0.565004,0.565693
100,0.329835,0.040076,0.430480,1.045892,1.052147,1.048486


##### Calculate weights time complexity

In [65]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.016249,0.018590,0.019644,0.012924,0.019512,0.016255
1,0.024193,0.017271,0.019348,0.019141,0.021121,0.022279
10,0.017785,0.020264,0.019207,0.013734,0.014345,0.017209
100,0.012907,0.017553,0.014360,0.017920,0.014780,0.018081
30,0.019894,0.018451,0.016424,0.017510,0.016753,0.017044
50,0.017682,0.015078,0.017862,0.018102,0.012056,0.019661


##### Finding subsequences time complexity

In [66]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.459904,0.054294,0.416157,0.855196,0.859261,0.824232
1,1.177601,0.095188,0.252778,0.618989,0.526851,0.577188
10,0.423497,0.029899,0.252129,0.469564,0.450308,0.544677
100,0.405179,0.031332,0.260453,0.742328,0.623679,0.737281
30,0.420143,0.030087,0.263972,0.655696,0.567569,0.628217
50,0.470745,0.024119,0.272634,0.636393,0.534400,0.671816


##### Transform Dataset time complexity

In [67]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,15.602972,1.192063,12.328646,31.121478,30.450258,30.270918
1,1.074954,0.195119,0.442162,0.938418,0.913739,0.937944
10,2.253403,0.494439,3.051443,7.219708,7.024177,7.133466
100,19.342963,2.186144,25.528242,62.103425,62.067875,62.326692
30,6.674336,1.016662,8.268483,20.009024,20.160730,20.298974
50,10.614272,1.596156,13.476441,33.260166,32.952595,33.180834


##### Classifier time complexity

In [68]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.033931,0.274020,0.018657,0.038802,0.039188,0.043833
1,0.378258,0.240628,0.250880,0.013227,0.245205,0.014150
10,0.014012,0.283628,0.014283,0.014773,0.012608,0.012744
100,0.026706,0.167848,0.023013,0.041625,0.043333,0.043195
30,0.011911,0.188222,0.014345,0.017251,0.019125,0.019168
50,0.018043,0.197961,0.016173,0.024397,0.018625,0.025202
